In [1]:
import os
import re

import black

In [2]:
def process_py_file(file_path):
    with open(file_path, "r", encoding="utf8") as f:
        file_content = f.read()

    # Remove import statements
    file_content = re.sub(r"^import.*$|^from .* import.*$", "", file_content, flags=re.MULTILINE)
    file_content = re.sub(r"from\s+\.\w+\s+import\s+[\w+,?\s?]+", "", file_content, flags=re.MULTILINE)
    file_content = re.sub(r"^sys.path.append.*\n", "", file_content, flags=re.MULTILINE)

    # Remove comments
    file_content = re.sub(r"#.*", "", file_content)

    # # Remove function descriptions and docstrings
    patron_docstrings_triple_comillas = r"(\"\"\")(.*?)(\"\"\")"
    patron_docstrings_comillas_simples = r"(\'\'\')(.*?)(\'\'\')"
    patron_total = f"{patron_docstrings_triple_comillas}|{patron_docstrings_comillas_simples}"

    file_content = re.sub(patron_total, "", file_content, flags=re.DOTALL)

    # Remove the code inside if __name__ == "__main__": block
    file_content = re.sub(r'if __name__ == "__main__":[\s\S]*', "", file_content)

    # Remove any empty lines and extra whitespaces
    file_content = re.sub(r"^\s*\n", "", file_content, flags=re.MULTILINE)
    file_content = re.sub(r"\n\s*\n", "\n", file_content)
    file_content = file_content.strip()

    return file_content

In [3]:
def get_import_lines(file_path):
    with open(file_path, "r", encoding="utf8") as f:
        file_content = f.read()

    # Find all import lines using regular expression
    import_lines = re.findall(r"^import.*$|^from .* import.*$", file_content, re.MULTILINE)

    return import_lines

In [4]:
input_file_path = "../../phitter/continuous/continuous_distributions/fatigue_life.py"
processed_content = process_py_file(input_file_path)
print(processed_content)

class FatigueLife:
    def __init__(
        self,
        parameters: dict[str, int | float] = None,
        continuous_measures=None,
        init_parameters_examples=False,
    ):
        if continuous_measures is None and parameters is None and init_parameters_examples == False:
            raise ValueError(
                "You must initialize the distribution by providing one of the following: distribution parameters, a Continuous Measures [ContinuousMeasures] instance, or by setting init_parameters_examples to True."
            )
        if continuous_measures != None:
            self.parameters = self.get_parameters(continuous_measures=continuous_measures)
        if parameters != None:
            self.parameters = parameters
        if init_parameters_examples:
            self.parameters = self.parameters_example
        self.gamma = self.parameters["gamma"]
        self.loc = self.parameters["loc"]
        self.scale = self.parameters["scale"]
    @property
    def name(s

In [5]:
input_file_path = "../../phitter/continuous/continuous_distributions/fatigue_life.py"
import_lines = get_import_lines(input_file_path)

In [6]:
IMPORTS = []
for file in os.listdir("../../phitter/continuous/continuous_distributions"):
    if ".py" in file and file != "__init__.py":
        import_lines = get_import_lines(f"../../phitter/continuous/continuous_distributions/{file}")
        IMPORTS.extend(import_lines)

input_file_path = "../../phitter/continuous/continuous_measures/CONTINUOUS_MEASURES.py"
import_lines = get_import_lines(input_file_path)
IMPORTS.extend(import_lines)
IMPORTS.append("import concurrent.futures")
IMPORTS.append("import typing")

In [7]:
CODE = "\n".join(sorted(list(set(IMPORTS)))) + "\n\n"
for file in os.listdir("../../phitter/continuous/continuous_distributions"):
    if ".py" in file:
        processed_content = process_py_file(f"../../phitter/continuous/continuous_distributions/{file}")
        CODE += processed_content + "\n\n"

In [8]:
input_file_path = "../../phitter/continuous/continuous_measures/CONTINUOUS_MEASURES.py"
measuerements_code = process_py_file(input_file_path)
CODE += measuerements_code + "\n\n"

input_file_path = "../../phitter/continuous/continuous_statistical_tests/continuous_test_chi_square.py"
evaluate_continuous_test_chi_square_code = process_py_file(input_file_path)
CODE += evaluate_continuous_test_chi_square_code + "\n\n"

input_file_path = "../../phitter/continuous/continuous_statistical_tests/continuous_test_kolmogorov_smirnov.py"
evaluate_continuous_test_kolmogorov_smirnov_code = process_py_file(input_file_path)
CODE += evaluate_continuous_test_kolmogorov_smirnov_code + "\n\n"

input_file_path = "../../phitter/continuous/continuous_statistical_tests/continuous_test_anderson_darling.py"
evaluate_continuous_test_anderson_darling_code = process_py_file(input_file_path)
evaluate_continuous_test_anderson_darling_code = evaluate_continuous_test_anderson_darling_code.replace("ad.", "")
CODE += evaluate_continuous_test_anderson_darling_code + "\n\n"

In [9]:
class_phitter_continuous_code = process_py_file("../../phitter/continuous/phitter_continuous.py")
CODE += class_phitter_continuous_code + "\n\n"

In [10]:
CODE = black.format_str(CODE, mode=black.FileMode(line_length=300))

In [11]:
code_file = open("./phitter_web_continuous.py", "+w", encoding="utf8")
code_file.write(CODE)
code_file.close()